In [1]:
import os
import pandas as pd
import numpy as np

from tqdm import tqdm
tqdm.pandas()

import requests
import re
import pprint

In [2]:
path = '../Data/Origin/사용자분석231204.csv'

sales_df_origin = pd.read_csv(path)

In [3]:
basic_column = ['사용자코드', '날짜', '요일', '성별', '나이', '사용자 주소', '가맹점 사업자 번호']

sales_df_origin = sales_df_origin.loc[(sales_df_origin[basic_column].isnull().sum(axis = 1) == 0)]

## 사용자 거주지 

### 과학기술정보통신부 우정사업본부_우편전용 우편번호조회
    * https://www.data.go.kr/iim/api/selectAPIAcountView.do
    * encoding key 목록
        - 1
            - d%2F2Eaf7qXnoQuZ7IwPJgXb92aDupDtGhVcLOmxffLcgDUC1wBD5afIWjgyxbjX8mKFcdWCOxOs%2BcngEAD6VYKA%3D%3D
            - d/2Eaf7qXnoQuZ7IwPJgXb92aDupDtGhVcLOmxffLcgDUC1wBD5afIWjgyxbjX8mKFcdWCOxOs+cngEAD6VYKA==        
        - 2
            - VqP4mQZ6yioAjqQWW%2FCPhgNZoDqZQuuHjveTDowjznZPpKbmyJ5K5yXC9wZWEAwZ%2FeJzjt3c1YcZEnuBhuH7tw%3D%3D
            - VqP4mQZ6yioAjqQWW/CPhgNZoDqZQuuHjveTDowjznZPpKbmyJ5K5yXC9wZWEAwZ/eJzjt3c1YcZEnuBhuH7tw==
        - 3
            - n47XHaApeo3Dyk2X4hO6MWiWHQMnmf12ZTED7q8xJ7ay1wBprvVxrwrsVze4z2xByi%2BhMMlFjhhR0wCeGIhHoA%3D%3D
            - n47XHaApeo3Dyk2X4hO6MWiWHQMnmf12ZTED7q8xJ7ay1wBprvVxrwrsVze4z2xByi+hMMlFjhhR0wCeGIhHoA==
        - 4
            - TSqYg2F8pDOoqchgJroQsp8JA79EMvzyf85hyv8x4wiKohry%2Fo2mn4%2BRiCd4NuDTkdCClXAA9cUaVwzJooQWmA%3D%3D
            - TSqYg2F8pDOoqchgJroQsp8JA79EMvzyf85hyv8x4wiKohry/o2mn4+RiCd4NuDTkdCClXAA9cUaVwzJooQWmA==
        - 5
            - NlYILeKCpotTBDOjX4LR%2FrmNmpYhQUba6ooyS7xpN%2BIFkZWupwzA3sqDsV9B7alcPTI8h894IupP4BiOTzxR9A%3D%3D
            - NlYILeKCpotTBDOjX4LR/rmNmpYhQUba6ooyS7xpN+IFkZWupwzA3sqDsV9B7alcPTI8h894IupP4BiOTzxR9A==
        - 6
            - F3carf84nddpXtSw%2BE4mLlrl4hdfroj8Z0y%2BrPhQrcyh8JsKIyvzAaoedGZT4NImmxeaSh2fU7rasxP1iSM2jg%3D%3D
            - F3carf84nddpXtSw+E4mLlrl4hdfroj8Z0y+rPhQrcyh8JsKIyvzAaoedGZT4NImmxeaSh2fU7rasxP1iSM2jg==
        - 7
            - Xr1y2g7%2BJIW9%2F8j%2FCkNxKCCGWbyQCYp%2B%2FrjgGfS%2B8GHiRUbYodo363awRtffSc26LP3z%2BQLLZADqZQxO%2B2MLuQ%3D%3D
            - Xr1y2g7+JIW9/8j/CkNxKCCGWbyQCYp+/rjgGfS+8GHiRUbYodo363awRtffSc26LP3z+QLLZADqZQxO+2MLuQ==

In [237]:
pmj = 'VqP4mQZ6yioAjqQWW/CPhgNZoDqZQuuHjveTDowjznZPpKbmyJ5K5yXC9wZWEAwZ/eJzjt3c1YcZEnuBhuH7tw=='
pyj = 'n47XHaApeo3Dyk2X4hO6MWiWHQMnmf12ZTED7q8xJ7ay1wBprvVxrwrsVze4z2xByi+hMMlFjhhR0wCeGIhHoA=='
ksb = 'TSqYg2F8pDOoqchgJroQsp8JA79EMvzyf85hyv8x4wiKohry/o2mn4+RiCd4NuDTkdCClXAA9cUaVwzJooQWmA=='
mjs = 'NlYILeKCpotTBDOjX4LR/rmNmpYhQUba6ooyS7xpN+IFkZWupwzA3sqDsV9B7alcPTI8h894IupP4BiOTzxR9A=='
ysh = 'F3carf84nddpXtSw+E4mLlrl4hdfroj8Z0y+rPhQrcyh8JsKIyvzAaoedGZT4NImmxeaSh2fU7rasxP1iSM2jg=='
ksh = 'Xr1y2g7+JIW9/8j/CkNxKCCGWbyQCYp+/rjgGfS+8GHiRUbYodo363awRtffSc26LP3z+QLLZADqZQxO+2MLuQ=='
ysw = 'd/2Eaf7qXnoQuZ7IwPJgXb92aDupDtGhVcLOmxffLcgDUC1wBD5afIWjgyxbjX8mKFcdWCOxOs+cngEAD6VYKA=='

In [71]:
people_df = sales_df_origin[['사용자코드', '사용자 주소', '시', '도']].drop_duplicates()
people_df['우편번호'] = np.nan
people_df['신규_사용자 주소'] = np.nan

In [238]:
def zipNo_address(keyword, serviceKey):
    keyword = re.sub('\([\w\W\S\s\d]*\)', '', keyword)
    keyword = keyword.replace(',', ' ').replace('  ', ' ')
    keyword = re.sub(' [-\da-zA-Z]*동', '', keyword)
    keyword = re.sub(' [-\d]*호', '', keyword)
    
    url = "http://openapi.epost.go.kr/postal/retrieveNewZipCdService/retrieveNewZipCdService/getNewZipCdList"
    params = {'serviceKey' : serviceKey,
              'srchwrd' : keyword,
              'countPerPage' : '10', 
              'currentPage' : '1'}
    response = requests.get(url, params = params)
    if response.ok != True:
        print('error :', keyword)
        return 'error', 'error'
    try:
        compile_address = re.compile('<address>[0-9\s\S\w\W]*</address>')
        address = compile_address.search(response.text)[0]
        address = address.replace('<address>', '').replace('</address>', '')

        compile_zipNo = re.compile('<zipNo>[0-9]*</zipNo>')
        zipNo = compile_zipNo.search(response.text)[0]
        zipNo = zipNo.replace('<zipNo>', '').replace('</zipNo>', '')
    except:
        return 'error', 'error'
    return address, zipNo

In [200]:
people_df['도'].unique()

array(['충남', nan, '충청남도', '경기', '강원특별자치도', '충북', '전북', '경북', '서울', '전남',
       '광주', '대전', '강원', '경남', '인천', '세종특별자치시', '경기도', '제주특별자치도', '부산'],
      dtype=object)

In [239]:
# tmp_df = people_df.iloc[:10000]
# serviceKey = pmj
# tmp_df[['신규_사용자 주소', '우편번호']] = tmp_df.progress_apply(lambda x : zipNo_address(x['사용자 주소'], serviceKey), axis = 1, result_type = 'expand')
# tmp_df.to_csv('../Data/Origin/사용자주소.csv', encoding = 'utf-8-sig', index = False)

# tmp_df = people_df.iloc[10000 : 20000]
# serviceKey = ksh
# tmp_df[['신규_사용자 주소', '우편번호']] = tmp_df.progress_apply(lambda x : zipNo_address(x['사용자 주소'], serviceKey), axis = 1, result_type = 'expand')
# tmp_df.to_csv('../Data/Origin/사용자주소.csv', encoding = 'utf-8-sig', index = False, header = None, mode = 'a')

# tmp_df = people_df.iloc[20000 : 30000]
# serviceKey = pyj
# tmp_df[['신규_사용자 주소', '우편번호']] = tmp_df.progress_apply(lambda x : zipNo_address(x['사용자 주소'], serviceKey), axis = 1, result_type = 'expand')
# tmp_df.to_csv('../Data/Origin/사용자주소.csv', encoding = 'utf-8-sig', index = False, header = None, mode = 'a')

# tmp_df = people_df.iloc[30000 : 40000]
# serviceKey = ksb
# tmp_df[['신규_사용자 주소', '우편번호']] = tmp_df.progress_apply(lambda x : zipNo_address(x['사용자 주소'], serviceKey), axis = 1, result_type = 'expand')
# tmp_df.to_csv('../Data/Origin/사용자주소.csv', encoding = 'utf-8-sig', index = False, header = None, mode = 'a')

# tmp_df = people_df.iloc[40000 : 50000]
# serviceKey = mjs
# tmp_df[['신규_사용자 주소', '우편번호']] = tmp_df.progress_apply(lambda x : zipNo_address(x['사용자 주소'], serviceKey), axis = 1, result_type = 'expand')
# tmp_df.to_csv('../Data/Origin/사용자주소.csv', encoding = 'utf-8-sig', index = False, header = None, mode = 'a')

# tmp_df = people_df.iloc[50000 : 60000]
# serviceKey = ysh
# tmp_df[['신규_사용자 주소', '우편번호']] = tmp_df.progress_apply(lambda x : zipNo_address(x['사용자 주소'], serviceKey), axis = 1, result_type = 'expand')
# tmp_df.to_csv('../Data/Origin/사용자주소.csv', encoding = 'utf-8-sig', index = False, header = None, mode = 'a')

# tmp_df = people_df.iloc[60000 : ]
# serviceKey = ysw
# tmp_df[['신규_사용자 주소', '우편번호']] = tmp_df.progress_apply(lambda x : zipNo_address(x['사용자 주소'], serviceKey), axis = 1, result_type = 'expand')
# tmp_df.to_csv('../Data/Origin/사용자주소.csv', encoding = 'utf-8-sig', index = False, header = None, mode = 'a')

100%|██████████| 3198/3198 [06:32<00:00,  8.14it/s]
C:\Users\WESLEYQUEST\AppData\Local\Temp\ipykernel_18560\103767058.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df[['신규_사용자 주소', '우편번호']] = tmp_df.progress_apply(lambda x : zipNo_address(x['사용자 주소'], serviceKey), axis = 1, result_type = 'expand')


## 부여군 읍/면 전처리

In [28]:
path = '../Data/Origin/사용자주소.csv'

df_user_origin = pd.read_csv(path)
df_user_origin['읍/면'] = np.nan

In [29]:
city_list = sorted([city if pd.isnull(city) == False else 'null' for city in df_user_origin['시'].unique()])

In [30]:
town_list = ['부여읍', '규암면', '은산면', '외산면', '내산면', '구룡면', '홍산면', '옥산면', '남면', '충화면', '양화면', '임천면', '장암면', '세도면', '석성면', '초촌면']

def findTown(x, town_list):
    for town in town_list:
        if town in x:
            return town
    
    return 'town 없음'

df_user_origin.loc[(df_user_origin['시'] == '부여군'), '읍/면'] = df_user_origin.loc[(df_user_origin['시'] == '부여군'), '사용자 주소'].apply(lambda x : findTown(x, town_list))

C:\Users\WESLEYQUEST\AppData\Local\Temp\ipykernel_21784\494669413.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['부여읍' '부여읍' '장암면' ... '규암면' '남면' '홍산면']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_user_origin.loc[(df_user_origin['시'] == '부여군'), '읍/면'] = df_user_origin.loc[(df_user_origin['시'] == '부여군'), '사용자 주소'].apply(lambda x : findTown(x, town_list))


In [31]:
condition01 = (df_user_origin['읍/면'] == 'town 없음')

print(len(df_user_origin.loc[condition01]))

df_user_origin.loc[condition01, '읍/면'] = df_user_origin.loc[condition01, '신규_사용자 주소'].apply(lambda x : findTown(x, town_list))

condition01 = (df_user_origin['읍/면'] == 'town 없음')

print(len(df_user_origin.loc[condition01]))

3232
154


### 부여군 외 전처리

In [33]:
# condition02 = (df_user_origin['시'] != '부여군') & (df_user_origin['시'].isnull() == True)
condition02 = (df_user_origin['시'] != '부여군') | (df_user_origin['시'].isnull() == True)

df_user_origin.loc[condition02 & (df_user_origin['사용자 주소'].str.contains('충남')), '도'] = '충남'
df_user_origin.loc[(df_user_origin['도'] == '충남') & (df_user_origin['시'].isnull() == True) & (df_user_origin['사용자 주소'].str.contains('부여군')), '시'] = '부여군'
df_user_origin.loc[(df_user_origin['시'] == '부여군') & (df_user_origin['읍/면'].isnull() == True), '읍/면'] = df_user_origin.loc[(df_user_origin['시'] == '부여군') & (df_user_origin['읍/면'].isnull() == True), '사용자 주소'].apply(lambda x : findTown(x, town_list))

In [34]:
condition03 = (df_user_origin['시'].isnull() == True)

df_user_origin.loc[condition03 & (df_user_origin['신규_사용자 주소'].str.contains('부여군')), '시'] = '부여군'
df_user_origin.loc[(df_user_origin['시'] == '부여군') & (df_user_origin['읍/면'].isnull() == True), '읍/면'] = df_user_origin.loc[(df_user_origin['시'] == '부여군') & (df_user_origin['읍/면'].isnull() == True), '신규_사용자 주소'].apply(lambda x : findTown(x, town_list))

In [35]:
df_user_origin.loc[(df_user_origin['시'] == '부여군') & (df_user_origin['읍/면'].isnull() == True)]

,사용자코드,사용자 주소,시,도,우편번호,신규_사용자 주소,읍/면


In [ ]:
# df_user_origin.to_csv('../Data/Preprocessed/사용자주소.csv', index = False, encoding = 'utf-8-sig')

## 사용데이터 수작업 내용 병합

In [53]:
path = '../Data/Origin/사용자주소수정용.xlsx'

df_emend_user_origin = pd.read_excel(path)
df_user_origin = pd.read_csv('../Data/Preprocessed/사용자주소.csv')

df_user_origin = df_user_origin.loc[~(df_user_origin['사용자코드'].isin(df_emend_user_origin['사용자코드']))]
df_user_origin = pd.concat([df_user_origin, df_emend_user_origin], axis = 0, ignore_index = True)
df_user_origin = df_user_origin.loc[(df_user_origin['시'] != '삭제')]

In [57]:
df_user_origin.loc[(df_user_origin['시'].isnull() == True)]

,사용자코드,사용자 주소,시,도,우편번호,신규_사용자 주소,읍/면


In [59]:
df_user_origin.loc[(df_user_origin['시'] == '부여군') & (df_user_origin['읍/면'].isnull() == True)]

,사용자코드,사용자 주소,시,도,우편번호,신규_사용자 주소,읍/면


In [60]:
df_user_origin.loc[(df_user_origin['시'] != '부여군') & (df_user_origin['읍/면'].isnull() != True)]

,사용자코드,사용자 주소,시,도,우편번호,신규_사용자 주소,읍/면


In [76]:
tmp_list = ['강남구', '서초구', '광산구', '동구', '서구', '중구',  
            '영등포구', '송파구', '강북구', '유성구', '동작구']

replace_city = {'청주시 흥덕구' : '청주시',
                '세종특별자치시' : '세종',
                '천안시 동남구' : '천안시',
                '천안시 서북구' : '천안시',
                '안산시 상록구' : '안산시',
                '창원시 의창구' : '창원시',
                '안양시 만안구' : '안양시',
                '청주시 청원구' : '청주시',
                '계룡' : '계룡시',
                '서천' : '서천군',
                '논산' : '논산시',
                '천안' : '천안시',
                '공주' : '공주시',
                '안산' : '안산시'
                }

df_user_origin['시'] = df_user_origin['시'].apply(lambda x : replace_city[x] if x in replace_city.keys() else x)
df_user_origin.loc[(df_user_origin['시'].isin(tmp_list)), '시'] = df_user_origin.loc[(df_user_origin['시'].isin(tmp_list)), '도']

In [77]:
df_user_origin.loc[(df_user_origin['시'] != '부여군'), '시'].unique()

array(['대전', '논산시', '청양군', '공주시', '수원시', '홍천군', '포천시', '서울', '광주', '화성시',
       '인천', '청주시', '김제시', '세종', '안산시', '경산시', '대구', '서천군', '익산시', '성남시',
       '서산시', '보령시', '군포시', '군산시', '홍성군', '계룡시', '정읍시', '천안시', '오산시',
       '아산시', '속초시', '울산', '평택시', '용인시', '안성시', '광명시', '남양주시', '전주시',
       '당진시', '남원시', '통영시', '원주시', '시흥시', '영동군', '김포시', '제천시', '여수시',
       '고양시', '하남시', '부산', '정선군', '금산군', '의정부시', '세종시', '옥천군', '과천시',
       '사천시', '구리시', '음성군', '여주시', '의왕시', '창원시', '나주시', '예천군', '양주시',
       '안양시', '완주군', '울릉군', '예산군', '임실군', '태안군', '안동시', '목포시', '영주시',
       '양산시', '거제시', '부안군', '김해시', '진천군', '보은군', '부천시', '밀양시', '이천시',
       '제주시', '광주시', '강릉시', '영암군', '파주시', '고흥군', '포항시', '함안군', '진주시',
       '산청군', '거창군', '고창군', '동두천시', '화천군', '무안군', '해남군', '구미시', '성주군',
       '춘천시', '양평군', '달성군', '무주군', '강동구', '상주시', '옹진군', '양구군', '경주시',
       '영광군', '서귀포시', '순창군', '기장군', '장흥군', '평창군', '동해시', '순천시', '함양군',
       '태백시', '증평군', '광양시', '강화군', '단양군', '사하구', '충주시', '울주군', '담양군'],
      dty

In [78]:
df_user_origin.loc[(df_user_origin['시'] == '부여군'), '읍/면'].unique()

array(['부여읍', '장암면', '규암면', '은산면', '초촌면', '외산면', '구룡면', '남면', '내산면',
       '석성면', '홍산면', '옥산면', '양화면', '임천면', '충화면', '세도면'], dtype=object)

In [84]:
df_user_origin.loc[(df_user_origin['도'].isnull() == True), '시'].unique()

array([], dtype=object)

In [83]:
replace_state_null = {'논산시' : '충남',
                      '용인시' : '경기',
                      '부여군' : '충남',
                      '세종시' : '세종',
                      '달성군' : '대구',
                      '서울' : '서울',
                      '대전' : '대전',
                      '광주' : '광주',
                      '인천' : '인천',
                      '세종' : '세종',
                      '울산' : '울산',
                      '부산' : '부산',
                      '옹진군' : '인천',
                      '양구군' : '강원',
                      '기장군' : '부산',
                      '평창군' : '강원',
                      '속초시' : '강원',
                      '태백시' : '강원',
                      '춘천시' : '춘천',
                      '강화군' : '인천',
                      '울주군' : '울산'}
    

df_user_origin.loc[(df_user_origin['도'].isnull() == True), '도'] = df_user_origin.loc[(df_user_origin['도'].isnull() == True), '시'].apply(lambda x : replace_state_null[x] if x in replace_state_null.keys() else x)

In [88]:
df_user_origin['도'].unique()

array(['충남', '대전', '경기', '강원', '서울', '광주', '인천', '충북', '전북', '세종', '경북',
       '대구', '전남', '울산', '경남', '부산', '제주'], dtype=object)

In [87]:
replace_state = {'충청남도' : '충남', 
                '강원특별자치도' : '강원', 
                '제주특별자치도' : '제주',
                '춘천' : '강원',
                '경기도' : '경기'
                }

df_user_origin['도'] = df_user_origin['도'].apply(lambda x : replace_state[x] if x in replace_state.keys() else x)

In [85]:
df_user_origin['시'].unique()

array(['부여군', '대전', '논산시', '청양군', '공주시', '수원시', '홍천군', '포천시', '서울', '광주',
       '화성시', '인천', '청주시', '김제시', '세종', '안산시', '경산시', '대구', '서천군', '익산시',
       '성남시', '서산시', '보령시', '군포시', '군산시', '홍성군', '계룡시', '정읍시', '천안시',
       '오산시', '아산시', '속초시', '울산', '평택시', '용인시', '안성시', '광명시', '남양주시',
       '전주시', '당진시', '남원시', '통영시', '원주시', '시흥시', '영동군', '김포시', '제천시',
       '여수시', '고양시', '하남시', '부산', '정선군', '금산군', '의정부시', '세종시', '옥천군',
       '과천시', '사천시', '구리시', '음성군', '여주시', '의왕시', '창원시', '나주시', '예천군',
       '양주시', '안양시', '완주군', '울릉군', '예산군', '임실군', '태안군', '안동시', '목포시',
       '영주시', '양산시', '거제시', '부안군', '김해시', '진천군', '보은군', '부천시', '밀양시',
       '이천시', '제주시', '광주시', '강릉시', '영암군', '파주시', '고흥군', '포항시', '함안군',
       '진주시', '산청군', '거창군', '고창군', '동두천시', '화천군', '무안군', '해남군', '구미시',
       '성주군', '춘천시', '양평군', '달성군', '무주군', '강동구', '상주시', '옹진군', '양구군',
       '경주시', '영광군', '서귀포시', '순창군', '기장군', '장흥군', '평창군', '동해시', '순천시',
       '함양군', '태백시', '증평군', '광양시', '강화군', '단양군', '사하구', '충주시', '울주군',
       '담양

In [ ]:
sales_df_origin = sales_df_origin.drop(columns = ['시', '도'])

sales_df_origin = sales_df_origin.merge(df_user_origin[['사용자코드', '시', '도', '읍/면']], how = 'right')

In [17]:
# 병합
path = '../Data/Preprocessed/사용자분석.csv'
sales_df_origin = pd.read_csv(path)

path = '../Data/Origin/가맹점분류(대분류위주 분석)_231118.xlsx'

df_franchise = pd.read_excel(path)
df_franchise = df_franchise.rename(columns = {'사업자번호' : '가맹점 사업자 번호'})

In [25]:
sales_df_origin = sales_df_origin[['사용자코드', '연도', '분기', '월', '날짜', '요일', '시간대명', '성별', '연령대', '나이', '사용자 주소', '가맹점 사업자 번호', '가맹점 사업자 업종', '사용금액', '시', '도', '읍/면']]
sales_df_origin = sales_df_origin.merge(df_franchise, on = '가맹점 사업자 번호', how = 'left')

10569620
10569620


In [29]:
# sales_df_origin.to_csv('../Data/Preprocessed/사용자분석.csv', index = False, encoding = 'utf-8-sig', escapechar = '\\')

In [2]:
# 가맹점 주소 전처리
sales_df_origin = pd.read_csv('../Data/Preprocessed/사용자분석.csv')

In [4]:
sales_df_origin.columns

Index(['사용자코드', '연도', '분기', '월', '날짜', '요일', '시간대명', '성별', '연령대', '나이',
       '사용자 주소', '가맹점 사업자 번호', '가맹점 사업자 업종', '사용금액', '시', '도', '읍/면', '가맹점명',
       '골목상권분류', '대분류', '중분류', '소분류', '가맹점 주소', '가맹가입일자'],
      dtype='object')

In [5]:
가맹점_오류 = ['천호대로', '능동로', '신당2길', '백일헌로']
sales_df_origin[sales_df_origin['가맹점 주소'].isin(가맹점_오류)]
#94개 제거 대상

sales_df_origin = sales_df_origin[~sales_df_origin['가맹점 주소'].isin(가맹점_오류)]
#가맹점 주소 오류인 데이터 삭제 코드

#가맹점 주소 대체 List
부여읍 = ['성왕로173번길', '정림로', '부여로', '중앙로13번길', '중앙로', '사비로72번길', '사비로', '사비로100번길', '석탑로', '성왕로328번길', '중앙로5번길','성왕로','신기정로','궁남로', '중앙로5번길', '부장대로', '흑천로', '왕릉로', '신동엽길', '성왕로161번길']
구룡면 = ['충의로', '흥수로']
규암면 = ['자온로', '수북로41번길', '백제문로304번길', '규아면', '반산1구', '수북로', '충절로2123번길','충절로']
임천면 = ['성흥로', '성흥로97번길']
장암면 = ['의자로1235번길', '의자로', '장암로']
석성면 = ['선사로']
홍산면 = ['홍산시장로']
양화면 = ['입포로48번길']

#가맹점 주소 대체 데이터 대체 완료 코드
sales_df_origin.loc[(sales_df_origin['가맹점 주소'].isin(부여읍)),['가맹점 주소']] = '부여읍'
sales_df_origin.loc[(sales_df_origin['가맹점 주소'].isin(구룡면)),['가맹점 주소']] = '구룡면'
sales_df_origin.loc[(sales_df_origin['가맹점 주소'].isin(규암면)),['가맹점 주소']] = '규암면'
sales_df_origin.loc[(sales_df_origin['가맹점 주소'].isin(임천면)),['가맹점 주소']] = '임천면'
sales_df_origin.loc[(sales_df_origin['가맹점 주소'].isin(장암면)),['가맹점 주소']] = '장암면'
sales_df_origin.loc[(sales_df_origin['가맹점 주소'].isin(석성면)),['가맹점 주소']] = '석성면'
sales_df_origin.loc[(sales_df_origin['가맹점 주소'].isin(홍산면)),['가맹점 주소']] = '홍산면'
sales_df_origin.loc[(sales_df_origin['가맹점 주소'].isin(양화면)),['가맹점 주소']] = '양화면'
print(sales_df_origin['가맹점 주소'].nunique())

#NA 데이터 삭제 코드
sales_df_origin = sales_df_origin[~sales_df_origin['가맹점 주소'].isna()]
print(sales_df_origin['가맹점 주소'].nunique())

16
16


In [6]:
sales_df_origin['가맹점 주소'].unique()

array(['부여읍', '은산면', '규암면', '홍산면', '임천면', '외산면', '구룡면', '석성면', '내산면',
       '세도면', '초촌면', '남면', '장암면', '양화면', '옥산면', '충화면'], dtype=object)

In [7]:
# sales_df_origin.to_csv('../Data/Preprocessed/사용자분석.csv', index = False, encoding = 'utf-8-sig', escapechar = '\\')

# 부여군 사람 추가 작업

In [18]:
path = '../Data/Preprocessed/사용자분석.csv'

df_origin = pd.read_csv(path)

In [ ]:
town_list = ['부여읍', '규암면', '은산면', '외산면', '내산면', '구룡면', '홍산면', '옥산면', '남면', '충화면', '양화면', '임천면', '장암면', '세도면', '석성면', '초촌면']

for town in town_list:
    df_origin.loc[(df_origin['사용자 주소'].str.contains(town)) & (df_origin['시'] != '부여군'), ['도', '시', '읍/면']] = ['충남', '부여군', town]
    df_origin.loc[(df_origin['사용자코드'] == '김태수01075530769'), '도'] = '경기'

# df_origin.to_csv('../Data/Preprocessed/사용자분석.csv', index = False, encoding = 'utf-8-sig', escapechar = '\\')

In [19]:
df_origin.loc[(df_origin['사용자 주소'].str.contains('부여') & (df_origin['시'] != '부여군')), '사용자코드'].nunique()

1